In [1]:
import pandas as pd
import plotly.express as px
import json

constituency_geojson = json.load(open('parliament.geojson'))

In [2]:
constituency_code_to_name = pd.read_csv("constituency_incomes.csv")[["code", "name"]]

In [3]:
constituency_weights = pd.read_csv('constituency_weights.csv')

In [4]:
from policyengine_uk import Microsimulation

sim = Microsimulation()

poverty = sim.calculate("in_poverty", period=2025).values

In [5]:
poverty_by_constituency = (poverty * constituency_weights.T).T.mean()

In [6]:
constituency_geojson['features'][0]['properties']

{'FID': 1,
 'PCON22CD': 'E14000530',
 'PCON22NM': 'Aldershot',
 'BNG_E': 484884,
 'BNG_N': 155126,
 'LONG': -0.7841,
 'LAT': 51.28895,
 'GlobalID': '1620033b-a722-4a92-b611-d49fd8f3ed2b'}

In [7]:
la_data = []
# Iterative over JSON
for i in range(len(constituency_geojson["features"])):
    # Extract local authority name
    la = constituency_geojson["features"][i]['properties']['PCON22CD']
    # Assign the local authority name to a new 'id' property for later linking to dataframe
    constituency_geojson["features"][i]['id'] = la
    # While I'm at it, append local authority name to a list to make some dummy data to test, along with i for a value to test on map
    la_data.append([la,i])

import pandas as pd

# turn dummy data into a dataframe
df = pd.DataFrame(la_data)
# update column names
df.columns = ['LA','Val']

In [19]:
incomes = pd.read_csv("constituency_incomes.csv")

import numpy as np

ENGLAND_CONSTITUENCY = "E14"
NI_CONSTITUENCY = "N06"
SCOTLAND_CONSTITUENCY = "S14"
WALES_CONSTITUENCY = "W07"

incomes = incomes[np.any([
    incomes["code"].str.contains(country_code) for country_code in [ENGLAND_CONSTITUENCY, NI_CONSTITUENCY, SCOTLAND_CONSTITUENCY, WALES_CONSTITUENCY]
], axis=0)]

df = pd.DataFrame({
    "code": poverty_by_constituency.index,
    "poverty_rate": poverty_by_constituency.values,
    "name": incomes.set_index("code").loc[poverty_by_constituency.index].name.values
})